In [5]:
import time,datetime,psutil,os,pandas as pd

class battery:
    """Created By Mubeen Ahmad\n\nThis program Moniter the Battery Percentage\n\n\n2 parameter are default \n\n1 setsec manual --> setsec = False\n\n2 last_previous --> last_previous_per= False"""
    
    def __init__(self,setsec=False,last_previous_per=False):
        
        self.t1 = None # time difference for t1
        self.t2 = None # time difference for t2

        # operation break if actual data are modified as default op_break False
        self.op_break = False

        # last_previous_per = True means when the program run 2nd time than set last percentage valaue
        self.last_previous_per = last_previous_per
        
        self.counter = 1 # set counter for change dictionary key 
        self.data = {}   # actual data load in dictionary
        self.pervious_per = psutil.sensors_battery().percent # check percentage for manage according to logic

        try:
            os.mkdir("./report") # create a folder if the folder are not exits
            
        except  FileExistsError:

            # catch error it the folder exits
            if os.path.exists('./report/battery.csv'):
                # check if the previous csv file are exists
                self.check_previous_data() # if file found than manage old data

            else:
                # check if the last_pervious_per true
                # means in this block previous data are not found but if the user set
                # set the parameter true than show the warning
                # and set last_previous_per = False

                if self.last_previous_per:
                    print("Warning Previous Data not found")
                    print("last_previous_per = False")
                    last_previous_per = False

        
        self.flag = None # set flag None used latter (use for increase decrease constant)
        self.time_difference = None #  used latter for (percentage difference)

        # use for access the value charger are connected or not 
        self.st = {False:"Charger Not Connected \U0001FAAB",True:"Charger Connected \U0001F50B"}
        
        self.setsec = setsec # use for user time sleep 
        self.current_per = None # use for compare % previous or current
        self.status = None # use with self.st for access dictionary value True , False
        
    # call info datetime
    def get_current_datetime(self):
        return datetime.datetime.now()


    # get the dae
    def get_date(self):
       
        info = self.get_current_datetime() 
        
        d = info.strftime("%d")
        m = info.strftime("%m")
        y = info.strftime("%Y")
        date = f"{d}-{m}-{y}"
        return date
    
    # get the time
    def get_time(self):

        info = self.get_current_datetime()
        
        h = info.strftime("%I")
        mins = info.strftime("%M")
        sec = info.strftime("%S")
        p = info.strftime("%p")
            
        cur_time = f"{h}:{mins} {p} {sec} Second"
        return cur_time

    # get the battery info
    def sensor(self):
        sen = psutil.sensors_battery()
        self.current_per = sen.percent 
        self.status = sen.power_plugged

    # apply the logics
    
    def logic(self):

        # if user not set the second
        
        if self.setsec == False:
            
            # set t2 for current time
            self.t2 = time.time()
            # call sensor info
            self.sensor()

        # now compare if the % increase
        if self.current_per > self.pervious_per:
            self.flag = "Increase" # set flag

            # if userset a sec
            if self.setsec:
                # than set timedifference 
                self.time_difference = self.current_per-self.pervious_per
            else:
                # if user not set the sec
                self.t2 = time.time() # than set self.t2 for current time
                self.time_difference = self.t2-self.t1  # than set self.t2 for current time
            
            self.write_data() # write the data in file
            self.pervious_per = self.current_per # rwerite previous% with currentper 

        # now compare if the % decrease
        elif self.current_per < self.pervious_per:
            self.flag = "Decrease" # than set timedifference 

            # if userset a sec
            if self.setsec:
                # than set timedifference 
                self.time_difference = self.pervious_per-self.current_per
            else:
                # if user not set the sec
                self.t2 = time.time() # than set self.t2 for current time
                self.time_difference = self.t2-self.t1 # than set self.t2 for current time

            self.write_data() # write the data in file
            self.pervious_per = self.current_per # rwerite previous% with currentper 

        else:
            # if the battery % are constant
            if self.setsec: 
                # if user set the sec
                # means this condition only execute it the user set sec
                
                self.flag = "Constant" # set flag constant
                self.time_difference = self.pervious_per-self.pervious_per # than set timedifference 

                # write data
                self.write_data()


    def write_data(self):

        C_status = self.st.get(self.status) # extract data from the dictionary

        self.data.update({self.counter:{
            "Date":self.get_date(),
            "Time":self.get_time(),
            "Action":self.flag,
            "Status":self.st.get(self.status),
            "Previous %":self.pervious_per,
            "Current %":self.current_per,
            "% Difference":self.time_difference,
            "Set Seconds":self.setsec
        }})

        # convert dictionary in dataframe
        df = pd.DataFrame(self.data).T # transpose data for better understand
        df.to_csv("./report/battery.csv",index=False) # save data in csv
        self.t1 = self.t2 #update t1
        
        self.counter += 1 # update counter

    # load previous data
    def check_previous_data(self):

        # load in pandas csv
        previous_data = pd.read_csv("./report/battery.csv")        
        
        # add columns for manage
        self.cols = ["Date","Time","Action","Status","Previous %","Current %","% Difference","Set Seconds"]
        
        try:
            # catch error if columns are not found

            # rearange columns
            previous_data = previous_data[self.cols]

            # if user set last % true
            if self.last_previous_per:

                # add last column Previous % value
                # ad set in previous per
                self.pervious_per = float(previous_data['Previous %'].tail(1).values[0])

            # loop in values (cols)
            for v in previous_data.values:
                # add empty dictionary in self.data like { {}}
                self.data[self.counter] = {}

                # now loop in columns values
                for pos,i in enumerate(v): # use enumerate for iterate value

                    # add data self.data { {}} --> {1:{column:i}}
                    self.data[self.counter][self.cols[pos]] = i
                    
                # update counter for next row
                self.counter += 1

    
        
        except KeyError:

            # if key error means cant access column name
            # than show warning and set operation break = true
            print("battery.csv file are modifed")
            self.op_break = True

        

    # looping for iteration continusly
    def looping(self):

        # if operation break false
        if self.op_break == False:
           
            # update t1
            self.t1 = time.time()

            # and iterate continusly in loop
            while True:

                # if user set the sec
                if self.setsec:

                    # than wait for x second
                    time.sleep(self.setsec)
                    self.sensor() # call the sensor
                    self.logic() # apply logic

                else:
                    # otherwise call the logic if the user not set the setsec
                    self.logic()


        else:
            # if operation break true means columns are modifed
            # bcz of keyerror
            
            print("\nPlease Check this Columns")
            print(self.cols) # show required columns

            # show message for delete previous data
            print("\nOtherwise remove previous data")
        

# 2 parameter are defaule

# 1 setsec manual --> setsec = False
# 2 last_previous --> last_previous_per= False

In [12]:
obj = battery

In [13]:
print(obj.__doc__)

Created By Mubeen Ahmad

This program Moniter the Battery Percentage


2 parameter are default 

1 setsec manual --> setsec = False

2 last_previous --> last_previous_per= False


# Demo

### use setsec parameter

In [18]:
# by default this parameter are false
obj(setsec=3).looping() 

# break the program forcefully

KeyboardInterrupt: 

### check the data

In [21]:
!dir report\

 Volume in drive C has no label.
 Volume Serial Number is F28A-DB85

 Directory of C:\Users\HF\Desktop\venv\Scripts\report

10/10/2023  12:06 AM    <DIR>          .
10/10/2023  12:06 AM    <DIR>          ..
10/10/2023  12:06 AM    <DIR>          .ipynb_checkpoints
10/10/2023  12:06 AM               456 battery.csv
               1 File(s)            456 bytes
               3 Dir(s)  202,616,508,416 bytes free


### load battery.csv file in pandas

In [22]:
pd.read_csv("./report/battery.csv")

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92,92,0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92,92,0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92,92,0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92,92,0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92,92,0,3


### use last_previous_per 

In [24]:
#### by default last_previous_per = False
obj(setsec=6,last_previous_per=True).looping()

# this time i use setsec = 6
# and last percentage is 92

# break the program

KeyboardInterrupt: 

### now again check data

In [25]:
pd.read_csv("./report/battery.csv")

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.0,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.0,6


#### in this file previous data are found
#### and here i use previous percentage for compare current %

### now i cant use last % parameter

In [26]:
obj(setsec=6).looping()


# break program

KeyboardInterrupt: 

#### now check again data

In [27]:
pd.read_csv("./report/battery.csv")

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.0,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.0,6
7,10-10-2023,12:16 AM 46 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6
8,10-10-2023,12:16 AM 52 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6


### now trying change columns

In [33]:
df = pd.read_csv("./report/battery.csv")
df

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.0,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.0,6
7,10-10-2023,12:16 AM 46 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6
8,10-10-2023,12:16 AM 52 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6


In [36]:
df.rename(columns={"Status":"Xtatus"},inplace=True)

In [37]:
df

,Date,Time,Action,Xtatus,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.0,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.0,6
7,10-10-2023,12:16 AM 46 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6
8,10-10-2023,12:16 AM 52 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6


## now rewrite the file

In [38]:
df.to_csv("./report/battery.csv",index=False)

### now again run the program

In [39]:
obj(setsec=4).looping()

battery.csv file are modifed

Please Check this Columns
['Date', 'Time', 'Action', 'Status', 'Previous %', 'Current %', '% Difference', 'Set Seconds']

Otherwise remove previous data


### now program not run until we cant change column name

In [41]:
df.head()

,Date,Time,Action,Xtatus,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3


In [43]:
df.rename(columns={"Xtatus":"Status"},inplace=True)

In [44]:
df

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.0,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.0,6
7,10-10-2023,12:16 AM 46 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6
8,10-10-2023,12:16 AM 52 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6


In [46]:
df.to_csv("./report/battery.csv",index=False)

### Now again run the program

In [47]:
obj(setsec=3).looping()

# break program

KeyboardInterrupt: 

### now check the data

In [48]:
pd.read_csv("./report/battery.csv")

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.0,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.0,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.0,6
7,10-10-2023,12:16 AM 46 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6
8,10-10-2023,12:16 AM 52 Second,Constant,Charger Not Connected 🪫,85.0,85,0.0,6
9,10-10-2023,12:26 AM 10 Second,Constant,Charger Not Connected 🪫,78.0,78,0.0,3


### now i cant use setsec parameter

In [49]:
obj().looping()

# break program


KeyboardInterrupt



#### now check data

In [50]:
pd.read_csv("./report/battery.csv")

,Date,Time,Action,Status,Previous %,Current %,% Difference,Set Seconds
0,10-10-2023,12:06 AM 39 Second,Constant,Charger Not Connected 🪫,92.0,92,0.000000,3
1,10-10-2023,12:06 AM 42 Second,Constant,Charger Not Connected 🪫,92.0,92,0.000000,3
2,10-10-2023,12:06 AM 45 Second,Constant,Charger Not Connected 🪫,92.0,92,0.000000,3
3,10-10-2023,12:06 AM 48 Second,Constant,Charger Not Connected 🪫,92.0,92,0.000000,3
4,10-10-2023,12:06 AM 51 Second,Constant,Charger Not Connected 🪫,92.0,92,0.000000,3
5,10-10-2023,12:12 AM 38 Second,Decrease,Charger Not Connected 🪫,92.0,88,4.000000,6
6,10-10-2023,12:12 AM 44 Second,Constant,Charger Not Connected 🪫,88.0,88,0.000000,6
7,10-10-2023,12:16 AM 46 Second,Constant,Charger Not Connected 🪫,85.0,85,0.000000,6
8,10-10-2023,12:16 AM 52 Second,Constant,Charger Not Connected 🪫,85.0,85,0.000000,6
9,10-10-2023,12:26 AM 10 Second,Constant,Charger Not Connected 🪫,78.0,78,0.000000,3


## For better Analysis Dont use setsec Parameter